In [1]:
!pip install --upgrade transformers

In [4]:
# Zuha Umar
# 20I-0603
import pandas as pd
import numpy as np
from google.colab import drive

In [5]:
drive.mount('/content/drive')
train_file_path = "/content/drive/My Drive/train.csv"
test_file_path = "/content/drive/My Drive/test.csv"
test_labels_file_path = "/content/drive/My Drive/test_label.csv"

Mounted at /content/drive


In [7]:
df_train = pd.read_csv(train_file_path)
print(df_train.columns)

Index(['timestamp_(min)', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
       'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13',
       'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18',
       'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23',
       'feature_24'],
      dtype='object')


In [8]:
df_train.head(10)

,timestamp_(min),feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24
0,0.0,0.732689,0.761748,0.606848,0.488746,0.424310,0.403609,0.519318,0.398792,0.451453,...,0.318797,0.451856,0.571500,0.469717,0.609883,0.008432,0.000000,0.481838,0.006536,0.138249
1,1.0,0.732799,0.761855,0.607133,0.488781,0.432008,0.410256,0.511364,0.402568,0.455657,...,0.321463,0.456123,0.562226,0.466533,0.629812,0.008432,0.000000,0.477218,0.006536,0.115207
2,2.0,0.732938,0.761594,0.606895,0.488791,0.418858,0.407724,0.488636,0.396526,0.456104,...,0.347219,0.456692,0.572002,0.487845,0.643598,0.006745,0.000000,0.492623,0.008715,0.092166
3,3.0,0.732893,0.761656,0.606478,0.488802,0.417896,0.404242,0.500000,0.405589,0.460020,...,0.361904,0.460532,0.563354,0.479512,0.644690,0.008432,0.000000,0.457064,0.008715,0.142857
4,4.0,0.732788,0.761573,0.606777,0.488800,0.421103,0.407407,0.511364,0.399547,0.458507,...,0.359767,0.458825,0.563354,0.448298,0.629948,0.006745,0.000000,0.472223,0.006536,0.170507
5,5.0,0.732654,0.761304,0.607815,0.488751,0.403143,0.402026,0.528409,0.383686,0.449682,...,0.357345,0.450149,0.577767,0.462758,0.626945,0.006745,0.000000,0.475962,0.013072,0.138249
6,6.0,0.732544,0.761418,0.607595,0.488748,0.425914,0.408357,0.528409,0.419562,0.453380,...,0.364608,0.453705,0.566612,0.475290,0.629402,0.006745,0.003559,0.487324,0.008715,0.129032
7,7.0,0.732575,0.761475,0.608642,0.488771,0.428480,0.415321,0.534091,0.411254,0.464791,...,0.307913,0.465083,0.567239,0.488204,0.631313,0.008432,0.003559,0.473797,0.010893,0.142857
8,8.0,0.732440,0.761603,0.608882,0.488807,0.427518,0.410890,0.517045,0.407100,0.464057,...,0.309998,0.464514,0.576889,0.493189,0.636364,0.008432,0.000000,0.481318,0.008715,0.147465
9,9.0,0.732574,0.761135,0.609090,0.488796,0.418217,0.414688,0.528409,0.402190,0.457764,...,0.318445,0.458256,0.574759,0.502262,0.627628,0.008432,0.000000,0.483109,0.006536,0.124424


In [9]:
print("Shape of train data:", df_train.shape)

Shape of train data: (132481, 26)


In [10]:
def generate_binary_noise_mask(num_samples, num_features, masked_proportion, mean_length):
    pm_to_m = 1 - 1 / mean_length
    pm_to_u = 1 / mean_length
    pu_to_m = (1 / mean_length) * (masked_proportion / (1 - masked_proportion))
    pu_to_u = 1 - (1 / mean_length) * (masked_proportion / (1 - masked_proportion))

    P = np.array([[pm_to_m, pm_to_u],
                  [pu_to_m, pu_to_u]])
    M = np.zeros((num_samples, num_features), dtype=int)
    state = np.random.choice([0, 1], size=num_samples)
    for i in range(1, num_samples):
        state[i] = np.random.choice([0, 1], p=P[state[i-1]])
    mask_lengths = np.random.geometric(p=1/mean_length, size=num_samples)
    for i, length in enumerate(mask_lengths):
        start_idx = np.random.randint(0, num_features - length + 1)
        end_idx = start_idx + length
        M[i, start_idx:end_idx] = 1

    return M

no_samples = len(df_train)
no_features = len(df_train.columns) - 1 # Ignore time stamp column
masked_proportion = 0.2
mean_length = 2

binary_noise_mask = generate_binary_noise_mask(no_samples, no_features, masked_proportion, mean_length)
print("Binary Noise Mask:", df_train.shape)

Binary Noise Mask: (132481, 26)


In [34]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class TransformerEncoder(layers.Layer):
    def __init__(self, num_heads, d_model, dff, input_data_shape):
        super(TransformerEncoder, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.dff = dff
        self.input_data_shape = input_data_shape
        self.embedding = layers.Dense(d_model)
        self.pos_encoding = self.positional_encoding()
        self.enc_layers = [self.create_encoder_layer() for _ in range(2)]

    def create_encoder_layer(self):
        return layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.d_model // self.num_heads)

    def positional_encoding(self):
        position = tf.range(self.input_data_shape[1], dtype=tf.float32)[:, tf.newaxis]
        angle_rads = 1 / tf.pow(10000, (2 * (tf.range(self.d_model, dtype=tf.float32) // 2)) / tf.cast(self.d_model, tf.float32))
        angle_rads = tf.reshape(angle_rads, (1, -1))  # Reshape to broadcast across positions

        # Compute sin and cos values for even and odd indices
        sin_indices = tf.math.sin(position * angle_rads[:, 0::2])
        cos_indices = tf.math.cos(position * angle_rads[:, 1::2])

        # Interleave sin and cos values
        pos_encoding = tf.stack([sin_indices, cos_indices], axis=2)
        pos_encoding = tf.reshape(pos_encoding, (1, -1, self.d_model))
        return pos_encoding


class Generator(Model):
    def __init__(self, num_heads, d_model, dff, input_data_shape, output_shape):
        super(Generator, self).__init__()
        self.input_data_shape = input_data_shape
        self.encoder = TransformerEncoder(num_heads, d_model, dff, input_data_shape)

    def call(self, inputs, binary_noise_mask):
        # Encode the input data using the Transformer-based encoder with binary noise mask
        encoded_data = self.encoder(inputs, binary_noise_mask)
        return encoded_data

# Define model parameters
num_heads = 4
d_model = 128
dff = 512
masked_proportion = 0.2

# Get input shape from df_train
input_shape = df_train.shape

# Instantiate the Generator model
generator = Generator(num_heads, d_model, dff, input_shape, input_shape)

# Encode the input data using the generator
encoded_data = generator(df_train.values, binary_noise_mask)
print(encoded_data)

tf.Tensor(
[[0.00000000e+00 7.32689381e-01 7.61747539e-01 ... 4.81838137e-01
  6.53594779e-03 1.38248846e-01]
 [1.00000000e+00 7.32798815e-01 7.61854529e-01 ... 4.77218270e-01
  6.53594779e-03 1.15207374e-01]
 [2.00000000e+00 7.32937992e-01 7.61593759e-01 ... 4.92622644e-01
  8.71459674e-03 9.21659023e-02]
 ...
 [1.32478000e+05 7.75366724e-01 9.09095883e-01 ... 1.52412444e-01
  8.71459674e-03 1.19815670e-01]
 [1.32479000e+05 7.75383413e-01 9.09189463e-01 ... 1.53408602e-01
  8.71459674e-03 1.01382487e-01]
 [1.32480000e+05 7.75373936e-01 9.09185112e-01 ... 1.73375100e-01
  8.71459674e-03 1.05990782e-01]], shape=(132481, 26), dtype=float32)


In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_data(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)
    # Assuming the time series data doesn't need to handle timestamps for model input
    features = data.drop(['timestamp_(min)'], axis=1, errors='ignore')  # drop non-feature columns

    # Normalize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Convert to PyTorch tensors
    tensor_data = torch.tensor(scaled_features, dtype=torch.float32)
    return tensor_data

# Load and preprocess data
data_tensor = load_data(train_file_path)

# Create a DataLoader
train_loader = DataLoader(data_tensor, batch_size=64, shuffle=True)

def generate_binary_noise_mask(num_samples, num_features, masked_proportion, mean_length):
    # Calculate transition probabilities
    pm_to_u = 1 / mean_length  # Mask to unmask
    pm_to_m = 1 - pm_to_u      # Mask to mask
    pu_to_m = pm_to_u * (masked_proportion / (1 - masked_proportion))  # Unmask to mask
    pu_to_u = 1 - pu_to_m      # Unmask to unmask

    # Generate binary noise mask matrix using Markov chain for each feature
    mask = np.zeros((num_samples, num_features), dtype=int)

    for feature in range(num_features):
        # Start unmasked
        state = np.random.choice([0, 1], p=[1 - masked_proportion, masked_proportion])
        mask[0, feature] = state

        for i in range(1, num_samples):
            if state == 1:  # If currently masked
                state = np.random.choice([0, 1], p=[pm_to_u, pm_to_m])
            else:  # If currently unmasked
                state = np.random.choice([0, 1], p=[pu_to_m, pu_to_u])
            mask[i, feature] = state

    return torch.tensor(mask, dtype=torch.float32)

def masked_dataloader(data_tensor, batch_size, masked_proportion, mean_length):
    # Generate masks for all data
    num_samples, num_features = data_tensor.size()
    mask = generate_binary_noise_mask(num_samples, num_features, masked_proportion, mean_length)
    masked_data = data_tensor * mask  # Element-wise multiplication to apply the mask

    # Create a DataLoader
    dataset = TensorDataset(masked_data, data_tensor)  # Using original data as target
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example usage
train_loader = masked_dataloader(data_tensor, batch_size=64, masked_proportion=0.2, mean_length=5)

class TransformerEncoder(nn.Module):
    def __init__(self, feature_size, num_heads, num_layers, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.encoder_layers = nn.TransformerEncoderLayer(d_model=feature_size, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layers, num_layers=num_layers)

    def forward(self, src):
        return self.transformer_encoder(src)

class MLPDecoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLPDecoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)
        self.fc2 = nn.Linear(output_dim, output_dim)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        return self.activation(self.fc2(x))

class Generator(nn.Module):
    def __init__(self, feature_size, num_heads, num_encoder_layers, output_dim):
        super(Generator, self).__init__()
        self.encoder = TransformerEncoder(feature_size, num_heads, num_encoder_layers)
        self.decoder = MLPDecoder(feature_size, output_dim)

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        return self.activation(self.fc2(x))

# Reconstruction Loss
def reconstruction_loss(reconstructed, original):
    return nn.functional.mse_loss(reconstructed, original)

# Set up models, optimizers
generator = Generator(feature_size=25, num_heads=5, num_encoder_layers=3, output_dim=25).to(device)
discriminator = Discriminator(input_dim=25).to(device)

g_optimizer = optim.Adam(generator.parameters(), lr=0.001)
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)

def train_epoch(generator, discriminator, loader, g_optimizer, d_optimizer, device):
    for masked_data, real_data in loader:
        masked_data, real_data = masked_data.to(device), real_data.to(device)

        # Forward pass through generator
        generated_data = generator(masked_data)

        # Discriminator training
        d_optimizer.zero_grad()
        real_pred = discriminator(real_data)
        fake_pred = discriminator(generated_data.detach())
        d_loss = -(torch.log(real_pred) + torch.log(1 - fake_pred)).mean()
        d_loss.backward()
        d_optimizer.step()

        # Generator training
        g_optimizer.zero_grad()
        fake_pred = discriminator(generated_data)
        g_loss = -torch.log(fake_pred).mean()
        rec_loss = nn.functional.mse_loss(generated_data, real_data)  # Reconstruction loss
        total_g_loss = g_loss + rec_loss
        total_g_loss.backward()
        g_optimizer.step()

# Driver Code
num_epochs = 2
for epoch in range(num_epochs):
    train_epoch(generator, discriminator, train_loader, g_optimizer, d_optimizer, device)
    print(f'Epoch {epoch+1}/{num_epochs} completed.')

print(train_loader)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/2 completed.
Epoch 2/2 completed.


In [49]:
def calculate_anomalies(data_loader, generator, threshold, device):
    anomalies = []
    generator.eval()
    with torch.no_grad():
        for data, _ in data_loader:
            data = data.to(device)
            reconstructed_data = generator(data)
            error = nn.functional.mse_loss(reconstructed_data, data, reduction='none')
            error = error.mean(dim=1)
            anomalies.extend((error > threshold).cpu().numpy())

    return anomalies

# Example usage for detecting anomalies
threshold = 0.5
anomalies_detected = calculate_anomalies(train_loader, generator, threshold, device)
print(f"Count of anomalies: {np.sum(anomalies_detected)}")

Count of anomalies: 0


In [47]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras import layers, Model

# # Data Augmentation Module
# class DataAugmentation:
#     def __init__(self, masked_proportion=0.2, mean_length=2):
#         self.masked_proportion = masked_proportion
#         self.mean_length = mean_length

#     def generate_binary_noise_mask(self, num_samples, num_features):
#         pm_to_m = 1 - 1 / self.mean_length
#         pm_to_u = 1 / self.mean_length
#         pu_to_m = (1 / self.mean_length) * (self.masked_proportion / (1 - self.masked_proportion))
#         pu_to_u = 1 - (1 / self.mean_length) * (self.masked_proportion / (1 - self.masked_proportion))

#         P = np.array([[pm_to_m, pm_to_u],
#                       [pu_to_m, pu_to_u]])
#         M = np.zeros((num_samples, num_features), dtype=int)
#         state = np.random.choice([0, 1], size=num_samples)
#         for i in range(1, num_samples):
#             state[i] = np.random.choice([0, 1], p=P[state[i-1]])
#         mask_lengths = np.random.geometric(p=1/self.mean_length, size=num_samples)
#         for i, length in enumerate(mask_lengths):
#             start_idx = np.random.randint(0, num_features - length + 1)
#             end_idx = start_idx + length
#             M[i, start_idx:end_idx] = 1

#         return M

# # Generator Module
# class TransformerEncoder(layers.Layer):
#     def __init__(self, num_heads, d_model, dff, input_size):
#         super(TransformerEncoder, self).__init__()
#         self.num_heads = num_heads
#         self.d_model = d_model
#         self.dff = dff
#         self.input_size = input_size
#         self.embedding = layers.Dense(d_model)
#         self.pos_encoding = self.positional_encoding()
#         self.enc_layers = [self.create_encoder_layer() for _ in range(2)]

#     def create_encoder_layer(self):
#         return layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.d_model // self.num_heads)

#     def positional_encoding(self):
#         position = tf.range(self.input_size[1], dtype=tf.float32)[:, tf.newaxis]
#         angle_rads = 1 / tf.pow(10000, (2 * (tf.range(self.d_model, dtype=tf.float32) // 2)) / tf.cast(self.d_model, tf.float32))
#         pos_encoding = position * angle_rads
#         pos_encoding[:, 0::2] = tf.sin(pos_encoding[:, 0::2])
#         pos_encoding[:, 1::2] = tf.cos(pos_encoding[:, 1::2])
#         pos_encoding = pos_encoding[tf.newaxis, ...]
#         return tf.cast(pos_encoding, tf.float32)

#     def call(self, inputs, binary_noise_mask):
#         x = self.embedding(inputs)
#         x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
#         x += self.pos_encoding[:, :tf.shape(x)[1], :]

#         # Apply binary noise mask element-wise
#         masked_x = tf.subtract(x, tf.multiply(x, binary_noise_mask))

#         for enc_layer in self.enc_layers:
#             masked_x = enc_layer(masked_x, masked_x)
#         return masked_x

# class Generator(Model):
#     def __init__(self, num_heads, d_model, dff, input_size):
#         super(Generator, self).__init__()
#         self.encoder = TransformerEncoder(num_heads, d_model, dff, input_size)

#     def call(self, inputs, binary_noise_mask):
#         # Encode the input data using the Transformer-based encoder with binary noise mask
#         encoded_data = self.encoder(inputs, binary_noise_mask)
#         return encoded_data

# # Discriminator Module
# class Discriminator(Model):
#     def __init__(self, input_size):
#         super(Discriminator, self).__init__()
#         self.flatten = layers.Flatten()
#         self.dense_1 = layers.Dense(64, activation='relu')
#         self.dense_2 = layers.Dense(1, activation='sigmoid')

#     def call(self, inputs):
#         x = self.flatten(inputs)
#         x = self.dense_1(x)
#         return self.dense_2(x)

# # Contrastive Learning Module
# class ContrastiveLoss(tf.keras.losses.Loss):
#     def __init__(self, temperature=1.0):
#         super(ContrastiveLoss, self).__init__()
#         self.temperature = temperature

#     def call(self, positive_pairs, negative_pairs):
#         cosine_similarity = tf.reduce_sum(tf.multiply(positive_pairs, negative_pairs), axis=-1)
#         softmax_scores = tf.nn.softmax(cosine_similarity / self.temperature, axis=-1)
#         loss = -tf.math.log(softmax_scores[:, 0])
#         return tf.reduce_mean(loss)

# num_samples, num_features = df_train.shape
# num_heads = 4
# d_model = 128
# dff = 512
# input_size = (num_samples, num_features)

# # Instantiate modules
# data_augmentation = DataAugmentation()
# generator = Generator(num_heads=num_heads, d_model=d_model, dff=dff, input_size=input_size)


TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment